In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import re
import operator
from pandas.core.common import flatten
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
#import pyhdb
from time import ctime

In [2]:
from datetime import datetime, timedelta
from collections import OrderedDict

In [3]:
from collections import Counter

In [4]:
df = pd.read_csv('C:/Users/rsure/Python Notebooks/LSTM note books/EY/Data Extraction/EY_FINAL_1.csv')
# df_2 = pd.read_csv('Query_Owned_COMM_IOT_Switching.csv')
# df_3 = pd.read_csv('Query_Owned_PUBLIC_SECTOR_IOT_Switching.csv')
# df_4 = pd.read_csv('Query_Owned_SMALL_IOT_Switching.csv')
# df_5 = pd.read_csv('Query_Owned_SERVICEPROVIDER_AND_OTHER_IOT_Switching.csv')

In [5]:
df.shape

(1943087, 20)

In [6]:
df['FISCAL_QUARTER_ID'].max()

20204

In [7]:
quarters =[20204]

In [8]:
df = df[~df['FISCAL_QUARTER_ID'].isin(quarters)]

In [9]:
df.shape

(1870031, 20)

In [10]:
df['ACTIVE_3YEARS'].describe()

count    1.870031e+06
mean     6.919797e-02
std      1.082836e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      8.333300e-02
max      1.000000e+00
Name: ACTIVE_3YEARS, dtype: float64

In [11]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [12]:
df.shape

(1870031, 19)

In [13]:
df['SAVM_GROUP_ID'].nunique()

20010

Create Collab Revenue Feature 

In [14]:
df.SITES.describe()

count    1.870031e+06
mean     3.244493e+00
std      4.957895e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      3.000000e+00
max      2.160000e+02
Name: SITES, dtype: float64

In [15]:
df.head()

,SAVM_GROUP_ID,SAVM_SCMS_NAME,SAVM_VERTICAL_MARKET_SEGMENT,BUSINESS_ENTITY,FISCAL_QUARTER_ID,ACTUAL_BOOKINGS,LIST_PRICE,QUANTITY,EY_FLAG,TENURE,PENETRATION,CUMM_BOOK_SAVM,CUMM_BOOK_SAVM_ARC,AVG_REVENUE_VERT,DISCOUNT,AVG_QUR_BOOKINGS,SITES,ACTIVE_3YEARS,BOUGHT_EY_BEFORE
0,203686542,COMMERCIAL,Manufacturing,Computing Systems,20154,0,0,0,0,3,8,344409,0,401701,0.0,0,1,0.0,0
1,203686542,COMMERCIAL,Manufacturing,Other,20154,0,0,0,0,3,8,685288,0,401701,0.0,0,1,0.0,0
2,203686542,COMMERCIAL,Manufacturing,Other,20162,0,0,0,0,5,11,685288,0,302453,0.0,0,1,0.0,0
3,203686542,COMMERCIAL,Manufacturing,Services ASS_L1,20154,0,0,0,0,3,8,706082,0,401701,0.0,0,1,0.0,0
4,203687728,COMMERCIAL,Manufacturing,Data Center Networking,20141,0,0,0,0,0,0,180321,0,72389,0.0,0,1,0.0,0


In [16]:
df.BUSINESS_ENTITY.unique()

array(['Computing Systems', 'Other', 'Services ASS_L1',
       'Data Center Networking', 'Enterprise Switching',
       'Service Provider Routing', 'Collaboration', 'IOT', 'Wireless',
       'Security', 'Enterprise Routing', 'Services TSS_L1', 'Meraki',
       'UNKNOWN', nan, 'SP Video Software and Solutions',
       'Monitoring and Analytics', 'SP Video Infrastructure'],
      dtype=object)

In [17]:
df_allrev_gb = df.groupby(['SAVM_GROUP_ID','FISCAL_QUARTER_ID'])['ACTUAL_BOOKINGS'].sum().reset_index()

In [18]:
df_colrev_gb = df[(df['BUSINESS_ENTITY']=='Computing Systems')| (df['BUSINESS_ENTITY']=='Data Center Networking')].groupby(['SAVM_GROUP_ID','FISCAL_QUARTER_ID'])['ACTUAL_BOOKINGS'].sum().reset_index()

In [19]:
df_colrev_gb.rename({'ACTUAL_BOOKINGS':'ACTUAL_BOOKINGS_COL'},axis=1,inplace=True)

In [20]:
df_colper = pd.merge(df_allrev_gb,df_colrev_gb,on=['SAVM_GROUP_ID','FISCAL_QUARTER_ID'])

In [21]:
df_colper['DataCenterRevenue'] = df_colper['ACTUAL_BOOKINGS_COL']/df_colper['ACTUAL_BOOKINGS']

In [22]:
df_colper.drop(['ACTUAL_BOOKINGS','ACTUAL_BOOKINGS_COL'],axis=1,inplace=True)

In [23]:
df = df.merge(df_colper,how='left',on=['SAVM_GROUP_ID','FISCAL_QUARTER_ID'])

In [24]:
df.head()

,SAVM_GROUP_ID,SAVM_SCMS_NAME,SAVM_VERTICAL_MARKET_SEGMENT,BUSINESS_ENTITY,FISCAL_QUARTER_ID,ACTUAL_BOOKINGS,LIST_PRICE,QUANTITY,EY_FLAG,TENURE,PENETRATION,CUMM_BOOK_SAVM,CUMM_BOOK_SAVM_ARC,AVG_REVENUE_VERT,DISCOUNT,AVG_QUR_BOOKINGS,SITES,ACTIVE_3YEARS,BOUGHT_EY_BEFORE,DataCenterRevenue
0,203686542,COMMERCIAL,Manufacturing,Computing Systems,20154,0,0,0,0,3,8,344409,0,401701,0.0,0,1,0.0,0,0.0
1,203686542,COMMERCIAL,Manufacturing,Other,20154,0,0,0,0,3,8,685288,0,401701,0.0,0,1,0.0,0,0.0
2,203686542,COMMERCIAL,Manufacturing,Other,20162,0,0,0,0,5,11,685288,0,302453,0.0,0,1,0.0,0,NaN
3,203686542,COMMERCIAL,Manufacturing,Services ASS_L1,20154,0,0,0,0,3,8,706082,0,401701,0.0,0,1,0.0,0,0.0
4,203687728,COMMERCIAL,Manufacturing,Data Center Networking,20141,0,0,0,0,0,0,180321,0,72389,0.0,0,1,0.0,0,0.0


In [25]:
df['DataCenterRevenue'].fillna(0,inplace=True)

End Feature creation

In [26]:
df[df['EY_FLAG']==1]['BUSINESS_ENTITY'].value_counts()

Collaboration               33205
Services TSS_L1             12179
Security                     3490
Wireless                     1358
Enterprise Routing           1311
Enterprise Switching          912
Data Center Networking        348
IOT                           185
Other                         176
Computing Systems             158
Services ASS_L1               146
UNKNOWN                       120
Monitoring and Analytics      113
Service Provider Routing       62
Meraki                         42
Name: BUSINESS_ENTITY, dtype: int64

In [27]:
savm_colab = df[((df['BUSINESS_ENTITY']=='Computing Systems') | (df['BUSINESS_ENTITY']=='Data Center Networking'))&(df['EY_FLAG']==1)]['SAVM_GROUP_ID'].tolist()

In [28]:
len(Counter(savm_colab).keys())

171

In [29]:
df.loc[df.SAVM_GROUP_ID.isin(savm_colab)].SAVM_GROUP_ID.nunique()

171

In [30]:
df_1 = df[df.SAVM_GROUP_ID.isin(savm_colab)]

In [31]:
df_1[df_1['EY_FLAG']==1]['BUSINESS_ENTITY'].value_counts()

Services TSS_L1             650
Collaboration               457
Security                    355
Data Center Networking      348
Wireless                    285
Enterprise Switching        267
Enterprise Routing          246
Computing Systems           158
Other                       122
IOT                          93
Services ASS_L1              91
UNKNOWN                      74
Service Provider Routing     62
Monitoring and Analytics     21
Meraki                        3
Name: BUSINESS_ENTITY, dtype: int64

In [32]:
df_1.shape

(31820, 20)

In [33]:
df_1.head()

,SAVM_GROUP_ID,SAVM_SCMS_NAME,SAVM_VERTICAL_MARKET_SEGMENT,BUSINESS_ENTITY,FISCAL_QUARTER_ID,ACTUAL_BOOKINGS,LIST_PRICE,QUANTITY,EY_FLAG,TENURE,PENETRATION,CUMM_BOOK_SAVM,CUMM_BOOK_SAVM_ARC,AVG_REVENUE_VERT,DISCOUNT,AVG_QUR_BOOKINGS,SITES,ACTIVE_3YEARS,BOUGHT_EY_BEFORE,DataCenterRevenue
77,203711281,COMMERCIAL,Manufacturing,Other,20162,0,0,0,0,5,11,9674818,0,302453,0.0,0,1,0.0,0,0.011625
88,203715197,COMMERCIAL,Manufacturing,Service Provider Routing,20151,0,0,0,0,0,0,12182672,0,326270,0.0,0,1,0.0,0,0.340023
1379,203786184,COMMERCIAL,Professional Services,Other,20161,0,0,0,0,4,7,2492817,0,95619,0.0,0,1,0.0,0,0.000000
1380,203786184,COMMERCIAL,Professional Services,Other,20162,0,0,0,0,5,7,2492817,0,117106,0.0,0,1,0.0,0,0.000000
1963,238384110,COMMERCIAL,Manufacturing,Enterprise Routing,20171,0,0,0,0,4,7,471945,0,237022,0.0,0,1,0.0,0,0.000000


In [34]:
df_1[((df_1['BUSINESS_ENTITY']=='Computing Systems') | (df_1['BUSINESS_ENTITY']=='Data Center Networking')) & (df_1['EY_FLAG']==1)].shape

(506, 20)

In [35]:
df_1['BUSINESS_ENTITY'] = np.where ( ((df_1['BUSINESS_ENTITY']=='Computing Systems') | (df_1['BUSINESS_ENTITY']=='Data Center Networking')) & (df_1['EY_FLAG']==1),'Reco_Prod',df_1['BUSINESS_ENTITY'])

In [36]:
df_1.SAVM_GROUP_ID.nunique()

171

In [37]:
# df_1.drop(['ARCHITECTURE_BE'],axis=1,inplace=True)

In [38]:
def remove_trailing(x):
    x.sort_values('FISCAL_QUARTER_ID',inplace=True)
    x.reset_index(inplace=True,drop=True)
    i=(x.BUSINESS_ENTITY.values == 'Reco_Prod').argmax()
    df_temp=x.head(i+1)
    return df_temp
    

In [39]:
savm_ids=df_1.SAVM_GROUP_ID.unique().tolist()

In [40]:
len(savm_ids)

171

In [41]:
l_df=[]

In [42]:
for i in savm_ids:
    df_temp=df_1.loc[df_1.SAVM_GROUP_ID==i]
    print(df_temp.SAVM_GROUP_ID.unique())
    l_df.append(remove_trailing(df_temp))

[203711281]
[203715197]
[203786184]
[238384110]
[203811794]
[203738952]
[203805558]
[203731602]
[203743287]
[203771886]
[203804029]
[203893780]
[203900225]
[229243846]
[203685109]
[203688395]
[203729778]
[203778295]
[203840348]
[203876310]
[203894195]
[203901826]
[207467425]
[210348525]
[213090026]
[253432641]
[203771540]
[203857386]
[253394317]
[254834113]
[257222636]
[270467802]
[283349746]
[283506606]
[283535544]
[203729711]
[203761500]
[203799393]
[203886994]
[208057335]
[203778788]
[203685075]
[203782597]
[203893779]
[283390025]
[203801835]
[203841554]
[203721341]
[203733148]
[203801799]
[203894396]
[277975461]
[203731715]
[203876747]
[203893701]
[203894557]
[276965252]
[203848711]
[203883059]
[268119064]
[203801807]
[203801820]
[208637723]
[208057352]
[222127765]
[281559040]
[281618740]
[203899291]
[283392657]
[208057324]
[283465951]
[276521303]
[203790841]
[203685101]
[203801814]
[203842876]
[203801817]
[203751115]
[203882786]
[203781663]
[203786768]
[203799378]
[235494000]
[277

In [43]:
df_revised=pd.concat(l_df,axis=0)

In [44]:
df_revised.to_csv('df_revised_without_trailing_DataCenter.csv')

In [45]:
df_revised=pd.read_csv('df_revised_without_trailing_DataCenter.csv')

In [46]:
def generate_security_revenue(row):
    if (row['BUSINESS_ENTITY'] == 'Computing Systems') | (row['BUSINESS_ENTITY'] == 'Data Center Networking') | (row['BUSINESS_ENTITY'] == 'Reco_Prod'):
        return row['ACTUAL_BOOKINGS']
    else :
        return 0

In [47]:
def generate_cumilative_features(df_temp):
    df_temp['DataCenter_Revenue']=df_temp.apply (lambda row: generate_security_revenue(row), axis=1)
    df_temp_1=pd.DataFrame(df_temp.groupby(['SAVM_GROUP_ID','FISCAL_QUARTER_ID'])['DataCenter_Revenue'].sum()).reset_index()
    df_temp_1['DataCenter_Revenue_cumulative']=df_temp_1['DataCenter_Revenue'].cumsum()
    df_temp_1['Year']=df_temp_1['FISCAL_QUARTER_ID'].astype(str).str[:4]
    df_temp_2=pd.DataFrame(df_temp_1.groupby('Year')['DataCenter_Revenue'].sum()).reset_index()
    df_temp_2.columns=['Year','DataCenter_Revenue_cumulative_last_year']
    df_temp_2['DataCenter_Revenue_cumulative_last_year']=df_temp_2['DataCenter_Revenue_cumulative_last_year'].shift(1)
    df_temp_3=df_temp_1.merge(df_temp_2,on='Year',how='left')
    df_temp_3.drop(['DataCenter_Revenue','Year'],axis=1,inplace=True)
    df_temp_3['DataCenter_Revenue_cumulative_last_year'].fillna(0,inplace=True)
    df_final=df_temp.merge(df_temp_3,on=['SAVM_GROUP_ID','FISCAL_QUARTER_ID'],how='left')
    df_final.drop('DataCenter_Revenue',axis=1,inplace=True)
    return df_final

In [48]:
df_revised_new=df_revised.groupby('SAVM_GROUP_ID').apply(lambda x: generate_cumilative_features(x))

In [49]:
df_revised_new.shape

(22097, 23)

In [50]:
df_revised.shape

(22097, 21)

In [51]:
df_1=df_revised_new.copy()

In [52]:
df_1.head()

Unnamed: 0  SAVM_GROUP_ID SAVM_SCMS_NAME  \
SAVM_GROUP_ID                                               
203684841     0           0      203684841     ENTERPRISE   
              1           1      203684841     ENTERPRISE   
              2           2      203684841     ENTERPRISE   
              3           3      203684841     ENTERPRISE   
              4           4      203684841     ENTERPRISE   

                SAVM_VERTICAL_MARKET_SEGMENT       BUSINESS_ENTITY  \
SAVM_GROUP_ID                                                        
203684841     0                       Retail       Services TSS_L1   
              1                       Retail              Security   
              2                       Retail  Enterprise Switching   
              3                       Retail     Computing Systems   
              4                       Retail    Enterprise Routing   

                 FISCAL_QUARTER_ID  ACTUAL_BOOKINGS  LIST_PRICE  QUANTITY  \
SAVM_GROUP_ID                                                               
203684841     0              20151           692957      997169       101   
              1              20151            25772      117500     50000   
              2              20151            17560           0        75   
              3              20151             2911        7464        10   
              4              20151           228243      522425       940   

                 EY_FLAG  ...  CUMM_BOOK_SAVM_ARC  AVG_REVENUE_VERT  DISCOUNT  \
SAVM_GROUP_ID             ...                                                   
203684841     0        0  ...              708815            341845       0.3   
              1        0  ...               25772            341845       0.8   
              2        0  ...               17560            341845  -17560.0   
              3        0  ...                2911            341845       0.6   
              4        0  ...              228243            341845       0.6   

                 AVG_QUR_BOOKINGS  SITES  ACTIVE_3YEARS  BOUGHT_EY_BEFORE  \
SAVM_GROUP_ID                                                               
203684841     0            692957      2            0.0                 0   
              1             25772      2            0.0                 0   
              2             17560      2            0.0                 0   
              3              2911      2            0.0                 0   
              4            228243      2            0.0                 0   

                 DataCenterRevenue  DataCenter_Revenue_cumulative  \
SAVM_GROUP_ID                                                       
203684841     0            0.00228                           2997   
              1            0.00228                           2997   
              2            0.00228                           2997   
              3            0.00228                           2997   
              4            0.00228                           2997   

                 DataCenter_Revenue_cumulative_last_year  
SAVM_GROUP_ID                                             
203684841     0                                      0.0  
              1                                      0.0  
              2                                      0.0  
              3                                      0.0  
              4                                      0.0  

[5 rows x 23 columns]

In [53]:
df_1.drop(['SAVM_GROUP_ID','Unnamed: 0'],axis=1,inplace=True)

In [54]:
df_1.reset_index(inplace=True)

In [55]:
df_1.drop('level_1',axis=1,inplace=True)

In [56]:
df_1.columns.tolist()

['SAVM_GROUP_ID',
 'SAVM_SCMS_NAME',
 'SAVM_VERTICAL_MARKET_SEGMENT',
 'BUSINESS_ENTITY',
 'FISCAL_QUARTER_ID',
 'ACTUAL_BOOKINGS',
 'LIST_PRICE',
 'QUANTITY',
 'EY_FLAG',
 'TENURE',
 'PENETRATION',
 'CUMM_BOOK_SAVM',
 'CUMM_BOOK_SAVM_ARC',
 'AVG_REVENUE_VERT',
 'DISCOUNT',
 'AVG_QUR_BOOKINGS',
 'SITES',
 'ACTIVE_3YEARS',
 'BOUGHT_EY_BEFORE',
 'DataCenterRevenue',
 'DataCenter_Revenue_cumulative',
 'DataCenter_Revenue_cumulative_last_year']

In [57]:
df_1.loc[df_1.LIST_PRICE==0,'DISCOUNT']=0

In [58]:
#df_1.BUSINESS_ENTITY.unique()

In [59]:
df_DataCenter_First = pd.DataFrame(df_1.loc[(df_1.BUSINESS_ENTITY=='Computing Systems')| (df_1.BUSINESS_ENTITY=='Data Center Networking') | (df_1.BUSINESS_ENTITY=='Reco_Prod'),:].groupby(['SAVM_GROUP_ID'])['TENURE'].min())

In [60]:
#df_Security_First.SAVM_GROUP_ID.nunique()

In [61]:
df_DataCenter_First.reset_index(drop=False,inplace=True)

In [62]:
df_DataCenter_First.columns = ['SAVM_GROUP_ID','DataCenter_TENURE']

In [63]:
df_DataCenter_First.SAVM_GROUP_ID.nunique()

171

In [64]:
df_1 = df_1.merge(df_DataCenter_First,on='SAVM_GROUP_ID',how='inner')

In [65]:
df_1['DataCenter_TENURE'] = df_1['TENURE'] - df_1['DataCenter_TENURE']

In [66]:
df_1.loc[df_1.DataCenter_TENURE<=0,'DataCenter_TENURE'] = 0

In [67]:
cat_columns = ['SAVM_VERTICAL_MARKET_SEGMENT','BUSINESS_ENTITY','SAVM_SCMS_NAME']
int_columns = ['SAVM_GROUP_ID', 'FISCAL_QUARTER_ID','ACTUAL_BOOKINGS','QUANTITY','TENURE','PENETRATION','CUMM_BOOK_SAVM',
               'CUMM_BOOK_SAVM_ARC','AVG_REVENUE_VERT','DISCOUNT',
       'AVG_QUR_BOOKINGS', 'SITES', 'ACTIVE_3YEARS', 'BOUGHT_EY_BEFORE',
       'DataCenterRevenue','DataCenter_Revenue_cumulative','DataCenter_Revenue_cumulative_last_year','DataCenter_TENURE']

In [68]:
for column in df_1[cat_columns]:
    df_1[column].fillna(df_1[column].mode()[0], inplace=True)

In [69]:
for column in df_1[int_columns]:
    df_1[column].fillna(df_1[column].median(), inplace=True)

In [70]:
np.sort(df_1['FISCAL_QUARTER_ID'].unique())

array([20121, 20122, 20123, 20124, 20131, 20132, 20133, 20134, 20141,
       20142, 20143, 20144, 20151, 20152, 20153, 20154, 20161, 20162,
       20163, 20164, 20171, 20172, 20173, 20174, 20181, 20182, 20183,
       20184, 20191, 20192, 20193, 20194, 20201, 20202, 20203],
      dtype=int64)

In [71]:
#df_1.drop('SAVM_SCMS_NAME',axis=1,inplace=True)

In [72]:
#df_1.head()

In [73]:
df_1['FISCAL_QUARTER_ID'] = df_1['FISCAL_QUARTER_ID'].astype(str)
df_1['DT'] = df_1['FISCAL_QUARTER_ID'].str[0:4] +str(0)+df_1['FISCAL_QUARTER_ID'].str[4] 
df_1['DT'] = df_1['DT'].astype(int)
del df_1['FISCAL_QUARTER_ID']
df_1['year_quarter'] = df_1['DT']
del df_1['DT']

In [74]:
df_2 = df_1[['SAVM_GROUP_ID','year_quarter','BUSINESS_ENTITY','CUMM_BOOK_SAVM_ARC']]

In [75]:
df_temp=pd.DataFrame(df_1.loc[df_1.BUSINESS_ENTITY=='Reco_Prod'].groupby('year_quarter')['SAVM_GROUP_ID'].nunique()).reset_index()

In [76]:
df_temp.columns=['year_quarter','SAVM_count']

In [77]:
df_temp['year']=df_temp['year_quarter'].astype('str').str[:4]

In [78]:
df_temp_1=pd.DataFrame(df_temp.groupby('year')['SAVM_count'].sum()).reset_index()

In [79]:
df_temp_1['shifted_savm_count']=df_temp_1['SAVM_count'].shift(1)

In [80]:
df_temp_final=df_temp.merge(df_temp_1,on='year',how='left')

In [81]:
df_temp_final

,year_quarter,SAVM_count_x,year,SAVM_count_y,shifted_savm_count
0,201503,1,2015,2,NaN
1,201504,1,2015,2,NaN
2,201604,7,2016,7,2.0
3,201701,1,2017,21,7.0
4,201702,1,2017,21,7.0
5,201703,5,2017,21,7.0
6,201704,14,2017,21,7.0
7,201801,3,2018,28,21.0
8,201802,4,2018,28,21.0
9,201803,8,2018,28,21.0


In [82]:
df_1_final=df_1.merge(df_temp_final[['year_quarter','shifted_savm_count']],on='year_quarter',how='left')

In [83]:
df_1_final['shifted_savm_count'].fillna(0,inplace=True)

In [84]:
df_1=df_1_final.copy()

In [85]:
df_dummified=pd.get_dummies(df_1,prefix=['Vert','BE','Seg'],columns=['SAVM_VERTICAL_MARKET_SEGMENT','BUSINESS_ENTITY','SAVM_SCMS_NAME'])

In [86]:
df_dummified.shape

(22097, 58)

In [87]:
df_dummified['year_quarter'].unique()

array([201501, 201502, 201503, 201504, 201601, 201602, 201603, 201604,
       201701, 201702, 201703, 201704, 201801, 201802, 201803, 201804,
       201901, 201902, 201903, 201904, 202001, 202002, 201402, 201403,
       201404, 201201, 201202, 201203, 201204, 201301, 201302, 202003,
       201401, 201303, 201304], dtype=int64)

In [88]:
df_dummified.head()

,SAVM_GROUP_ID,ACTUAL_BOOKINGS,LIST_PRICE,QUANTITY,EY_FLAG,TENURE,PENETRATION,CUMM_BOOK_SAVM,CUMM_BOOK_SAVM_ARC,AVG_REVENUE_VERT,...,BE_Security,BE_Service Provider Routing,BE_Services ASS_L1,BE_Services TSS_L1,BE_UNKNOWN,BE_Wireless,Seg_COMMERCIAL,Seg_ENTERPRISE,Seg_PUBLIC SECTOR,Seg_SERVICE PROVIDER
0,203684841,692957,997169,101,0,0,0,67648766,708815,341845,...,0,0,0,1,0,0,0,1,0,0
1,203684841,25772,117500,50000,0,0,0,42756085,25772,341845,...,1,0,0,0,0,0,0,1,0,0
2,203684841,17560,0,75,0,0,0,26064854,17560,341845,...,0,0,0,0,0,0,0,1,0,0
3,203684841,2911,7464,10,0,0,0,3723673,2911,341845,...,0,0,0,0,0,0,0,1,0,0
4,203684841,228243,522425,940,0,0,0,14725600,228243,341845,...,0,0,0,0,0,0,0,1,0,0


In [89]:
df_dummified.shape

(22097, 58)

In [90]:
#df_dummified.columns.tolist()

In [91]:
#pl_columns=[col for col in df_dummified if col.startswith('Prod_')]
#pl_columns = ['Prod_Reco_Prod']
vr_columns=[col for col in df_dummified if col.startswith('Vert_')]
be_columns=[col for col in df_dummified if col.startswith('BE_')]
scms_columns=[col for col in df_dummified if col.startswith('Seg_')]
numerical_columns =  ['TENURE','PENETRATION','CUMM_BOOK_SAVM','AVG_REVENUE_VERT',
                     'DISCOUNT','AVG_QUR_BOOKINGS', 'SITES', 'ACTIVE_3YEARS', 'BOUGHT_EY_BEFORE','DataCenterRevenue','DataCenter_Revenue_cumulative','DataCenter_Revenue_cumulative_last_year','DataCenter_TENURE','shifted_savm_count']
sum_cols = ['ACTUAL_BOOKINGS','QUANTITY']

In [92]:
df_temp_1=df_dummified.groupby(['SAVM_GROUP_ID','year_quarter'])[vr_columns].apply(lambda x : x.astype(int).sum())

In [93]:
for column in df_temp_1.columns:
    df_temp_1[column]= np.where(df_temp_1[column]>0,1,0)

In [94]:
df_temp_2=df_dummified.groupby(['SAVM_GROUP_ID','year_quarter'])[be_columns].apply(lambda x : x.astype(int).sum())

In [95]:
for column in df_temp_2.columns:
    df_temp_2[column]= np.where(df_temp_2[column]>0,1,0)

In [96]:
df_temp_3 =  df_dummified.groupby(['SAVM_GROUP_ID','year_quarter'])[numerical_columns].max()
df_temp_4 =  df_dummified.groupby(['SAVM_GROUP_ID','year_quarter'])[sum_cols].sum()
df_temp_5=df_dummified.groupby(['SAVM_GROUP_ID','year_quarter'])[scms_columns].apply(lambda x : x.astype(int).sum())

In [97]:
for column in df_temp_5.columns:
    df_temp_5[column]= np.where(df_temp_5[column]>0,1,0)

In [98]:
df_aggregated=pd.concat([df_temp_1,df_temp_2,df_temp_3,df_temp_4,df_temp_5],axis=1)

In [99]:
df_aggregated.head()

Vert_Education- Public/Private  \
SAVM_GROUP_ID year_quarter                                   
203684841     201501                                     0   
              201502                                     0   
              201503                                     0   
              201504                                     0   
              201601                                     0   

                            Vert_Energy/Utilities  Vert_Financial Services  \
SAVM_GROUP_ID year_quarter                                                   
203684841     201501                            0                        0   
              201502                            0                        0   
              201503                            0                        0   
              201504                            0                        0   
              201601                            0                        0   

                            Vert_Government  Vert_Health Care  \
SAVM_GROUP_ID year_quarter                                      
203684841     201501                      0                 0   
              201502                      0                 0   
              201503                      0                 0   
              201504                      0                 0   
              201601                      0                 0   

                            Vert_Hospitality/Hotels & Leisure  \
SAVM_GROUP_ID year_quarter                                      
203684841     201501                                        0   
              201502                                        0   
              201503                                        0   
              201504                                        0   
              201601                                        0   

                            Vert_Manufacturing  Vert_Media/Entertainment  \
SAVM_GROUP_ID year_quarter                                                 
203684841     201501                         0                         0   
              201502                         0                         0   
              201503                         0                         0   
              201504                         0                         0   
              201601                         0                         0   

                            Vert_Professional Services  Vert_Retail  ...  \
SAVM_GROUP_ID year_quarter                                           ...   
203684841     201501                                 0            1  ...   
              201502                                 0            1  ...   
              201503                                 0            1  ...   
              201504                                 0            1  ...   
              201601                                 0            1  ...   

                            DataCenter_Revenue_cumulative  \
SAVM_GROUP_ID year_quarter                                  
203684841     201501                                 2997   
              201502                               738703   
              201503                               809211   
              201504                               820888   
              201601                              1064844   

                            DataCenter_Revenue_cumulative_last_year  \
SAVM_GROUP_ID year_quarter                                            
203684841     201501                                            0.0   
              201502                                            0.0   
              201503                                            0.0   
              201504                                            0.0   
              201601                                       820888.0   

                            DataCenter_TENURE  shifted_savm_count  \
SAVM_GROUP_ID year_quarter                                          
2036

In [100]:
df_aggregated.reset_index(inplace=True)

In [101]:
df_aggregated.head()

,SAVM_GROUP_ID,year_quarter,Vert_Education- Public/Private,Vert_Energy/Utilities,Vert_Financial Services,Vert_Government,Vert_Health Care,Vert_Hospitality/Hotels & Leisure,Vert_Manufacturing,Vert_Media/Entertainment,...,DataCenter_Revenue_cumulative,DataCenter_Revenue_cumulative_last_year,DataCenter_TENURE,shifted_savm_count,ACTUAL_BOOKINGS,QUANTITY,Seg_COMMERCIAL,Seg_ENTERPRISE,Seg_PUBLIC SECTOR,Seg_SERVICE PROVIDER
0,203684841,201501,0,0,0,0,0,0,0,0,...,2997,0.0,0,0.0,1314730,54577,0,1,0,0
1,203684841,201502,0,0,0,0,0,0,0,0,...,738703,0.0,1,0.0,1884522,9885,0,1,0,0
2,203684841,201503,0,0,0,0,0,0,0,0,...,809211,0.0,2,0.0,287263,28546,0,1,0,0
3,203684841,201504,0,0,0,0,0,0,0,0,...,820888,0.0,3,0.0,10508438,108124,0,1,0,0
4,203684841,201601,0,0,0,0,0,0,0,0,...,1064844,820888.0,4,0.0,1076009,88,0,1,0,0


In [102]:
df_aggregated.shape

(2625, 55)

In [103]:
df_aggregated.SAVM_GROUP_ID.nunique()

171

In [104]:
df_2.head()

,SAVM_GROUP_ID,year_quarter,BUSINESS_ENTITY,CUMM_BOOK_SAVM_ARC
0,203684841,201501,Services TSS_L1,708815
1,203684841,201501,Security,25772
2,203684841,201501,Enterprise Switching,17560
3,203684841,201501,Computing Systems,2911
4,203684841,201501,Enterprise Routing,228243


In [105]:
df2_pivot = df_2.pivot_table(values='CUMM_BOOK_SAVM_ARC',index=['SAVM_GROUP_ID', 'year_quarter'],
                 columns='BUSINESS_ENTITY',aggfunc=np.max).add_prefix('BEval_').reset_index()

In [106]:
df2_pivot.fillna(0,inplace=True)

In [107]:
df2_pivot.head()

BUSINESS_ENTITY,SAVM_GROUP_ID,year_quarter,BEval_Collaboration,BEval_Computing Systems,BEval_Data Center Networking,BEval_Enterprise Routing,BEval_Enterprise Switching,BEval_IOT,BEval_Meraki,BEval_Monitoring and Analytics,BEval_Other,BEval_Reco_Prod,BEval_SP Video Infrastructure,BEval_SP Video Software and Solutions,BEval_Security,BEval_Service Provider Routing,BEval_Services ASS_L1,BEval_Services TSS_L1,BEval_UNKNOWN,BEval_Wireless
0,203684841,201501,163481.0,2911.0,86.0,228243.0,17560.0,0.0,0.0,0.0,0.0,0.0,0.0,182560.0,25772.0,153.0,0.0,708815.0,0.0,1007.0
1,203684841,201502,332481.0,5822.0,732906.0,335638.0,220677.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25453.0,4237.0,3568.0,776040.0,0.0,579895.0
2,203684841,201503,335961.0,80468.0,732906.0,345999.0,413355.0,0.0,0.0,0.0,0.0,0.0,0.0,372422.0,26051.0,4193.0,3568.0,857153.0,0.0,613614.0
3,203684841,201504,330019.0,80443.0,740544.0,3506090.0,2223217.0,37597.0,0.0,0.0,0.0,0.0,0.0,372422.0,62115.0,4193.0,-92139.0,3706744.0,0.0,3025706.0
4,203684841,201601,398774.0,0.0,984401.0,3645384.0,2263772.0,36595.0,0.0,0.0,0.0,0.0,0.0,473800.0,62115.0,5314.0,-92139.0,4708604.0,0.0,3043443.0


In [108]:
df_2.loc[(df_2.SAVM_GROUP_ID==203684841)&(df_2.year_quarter==201502),:]

,SAVM_GROUP_ID,year_quarter,BUSINESS_ENTITY,CUMM_BOOK_SAVM_ARC
10,203684841,201502,Data Center Networking,732906
11,203684841,201502,Services TSS_L1,776040
12,203684841,201502,Wireless,579895
13,203684841,201502,Service Provider Routing,4237
14,203684841,201502,Security,25453
15,203684841,201502,Services ASS_L1,3568
16,203684841,201502,Enterprise Routing,335638
17,203684841,201502,Computing Systems,5822
18,203684841,201502,Collaboration,332481
19,203684841,201502,Enterprise Switching,220677


In [109]:
#df2_pivot.loc[df2_pivot.SAVM_GROUP_ID==203684841,:]

In [110]:
df2_pivot.shape

(2625, 20)

In [111]:
df_aggregated_1 = pd.merge(df_aggregated,df2_pivot,on=['SAVM_GROUP_ID','year_quarter'])

In [112]:
df_aggregated_1.columns.tolist()

['SAVM_GROUP_ID',
 'year_quarter',
 'Vert_Education- Public/Private',
 'Vert_Energy/Utilities',
 'Vert_Financial Services',
 'Vert_Government',
 'Vert_Health Care',
 'Vert_Hospitality/Hotels & Leisure',
 'Vert_Manufacturing',
 'Vert_Media/Entertainment',
 'Vert_Professional Services',
 'Vert_Retail',
 'Vert_Service Provider',
 'Vert_Technical Services',
 'Vert_Transportation',
 'Vert_Unknown',
 'Vert_Wholesale/Distribution',
 'BE_Collaboration',
 'BE_Computing Systems',
 'BE_Data Center Networking',
 'BE_Enterprise Routing',
 'BE_Enterprise Switching',
 'BE_IOT',
 'BE_Meraki',
 'BE_Monitoring and Analytics',
 'BE_Other',
 'BE_Reco_Prod',
 'BE_SP Video Infrastructure',
 'BE_SP Video Software and Solutions',
 'BE_Security',
 'BE_Service Provider Routing',
 'BE_Services ASS_L1',
 'BE_Services TSS_L1',
 'BE_UNKNOWN',
 'BE_Wireless',
 'TENURE',
 'PENETRATION',
 'CUMM_BOOK_SAVM',
 'AVG_REVENUE_VERT',
 'DISCOUNT',
 'AVG_QUR_BOOKINGS',
 'SITES',
 'ACTIVE_3YEARS',
 'BOUGHT_EY_BEFORE',
 'DataCen

In [113]:
df_aggregated_1.drop('BEval_Reco_Prod',axis=1,inplace=True)

In [114]:
df_aggregated_1.head()

,SAVM_GROUP_ID,year_quarter,Vert_Education- Public/Private,Vert_Energy/Utilities,Vert_Financial Services,Vert_Government,Vert_Health Care,Vert_Hospitality/Hotels & Leisure,Vert_Manufacturing,Vert_Media/Entertainment,...,BEval_Monitoring and Analytics,BEval_Other,BEval_SP Video Infrastructure,BEval_SP Video Software and Solutions,BEval_Security,BEval_Service Provider Routing,BEval_Services ASS_L1,BEval_Services TSS_L1,BEval_UNKNOWN,BEval_Wireless
0,203684841,201501,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,182560.0,25772.0,153.0,0.0,708815.0,0.0,1007.0
1,203684841,201502,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,25453.0,4237.0,3568.0,776040.0,0.0,579895.0
2,203684841,201503,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,372422.0,26051.0,4193.0,3568.0,857153.0,0.0,613614.0
3,203684841,201504,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,372422.0,62115.0,4193.0,-92139.0,3706744.0,0.0,3025706.0
4,203684841,201601,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,473800.0,62115.0,5314.0,-92139.0,4708604.0,0.0,3043443.0


In [115]:
df_aggregated_final=df_aggregated_1.groupby(['SAVM_GROUP_ID']).apply(lambda x: x.sort_values('year_quarter',ascending=True))

In [116]:
df_aggregated_final.reset_index(inplace=True,drop=True)

In [117]:
df_aggregated_final.shape

(2625, 72)

In [118]:
# df_1=[]
# df=[]
# df_all=[]
# df_dummified=[]
# df_aggregated=[]
# df_temp_1=[]
# df_temp_2=[]
# df_temp_3=[]
# df_temp_4=[]
# df_temp_5=[]
# df_temp_6=[]

In [119]:
def generate_qtrs(min_,max_):
    l_qtrs=[]
    yrqtr=0
    l_qtrs.append(min_)
    while yrqtr!=max_:
        qtr = int(str(int(min_))[-1:])
        year = int(str(int(min_))[:4])
        if(qtr==4):
            year = year+1
            yrqtr = str(year)+'01'
            yrqtr = int(yrqtr)
        else:
            yrqtr = int(min_)+1 
        #print(yrqtr)    
        l_qtrs.append(yrqtr)
        min_=yrqtr
    return l_qtrs

In [120]:
def insert_rows(df):
    df.reset_index(inplace=True,drop=True)
    df.sort_values('year_quarter',inplace=True)
    if df.year_quarter.nunique()==1:
        return df
    else :
        dates=[df.year_quarter.min(),df.year_quarter.max()]
        #df.drop_duplicates('year_quarter',inplace=True)
        min_=df.year_quarter.min()
        max_=df.year_quarter.max()
        qtrs_list=generate_qtrs(min_,max_)
        full_idx=qtrs_list
    #df['year_quarter']=df['year_quarter'].astype('str')
        df.set_index('year_quarter',inplace=True)
    #print(df)
        df_=df.reindex(full_idx)
    #print(df_)
        len_=df_.shape[0]
        first_value=0
        l=[]
        cnt_=0
        for i in range(len_):
            l.append(first_value+cnt_)
            cnt_=cnt_+1
        df_['TENURE']=l
        df_.fillna(method='ffill',inplace=True)
        df_.reset_index(inplace=True)
        dates=[df_.year_quarter.min(),df_.year_quarter.max()]
        return df_ 

In [121]:
df_aggregated_inserted = df_aggregated_final.groupby(['SAVM_GROUP_ID']).apply(lambda x: insert_rows(x))

In [122]:
#df_aggregated_inserted.SAV

In [123]:
df_aggregated_inserted.head()

year_quarter  SAVM_GROUP_ID  Vert_Education- Public/Private  \
SAVM_GROUP_ID                                                                  
203684841     0        201501    203684841.0                             0.0   
              1        201502    203684841.0                             0.0   
              2        201503    203684841.0                             0.0   
              3        201504    203684841.0                             0.0   
              4        201601    203684841.0                             0.0   

                 Vert_Energy/Utilities  Vert_Financial Services  \
SAVM_GROUP_ID                                                     
203684841     0                    0.0                      0.0   
              1                    0.0                      0.0   
              2                    0.0                      0.0   
              3                    0.0                      0.0   
              4                    0.0                      0.0   

                 Vert_Government  Vert_Health Care  \
SAVM_GROUP_ID                                        
203684841     0              0.0               0.0   
              1              0.0               0.0   
              2              0.0               0.0   
              3              0.0               0.0   
              4              0.0               0.0   

                 Vert_Hospitality/Hotels & Leisure  Vert_Manufacturing  \
SAVM_GROUP_ID                                                            
203684841     0                                0.0                 0.0   
              1                                0.0                 0.0   
              2                                0.0                 0.0   
              3                                0.0                 0.0   
              4                                0.0                 0.0   

                 Vert_Media/Entertainment  ...  \
SAVM_GROUP_ID                              ...   
203684841     0                       0.0  ...   
              1                       0.0  ...   
              2                       0.0  ...   
              3                       0.0  ...   
              4                       0.0  ...   

                 BEval_Monitoring and Analytics  BEval_Other  \
SAVM_GROUP_ID                                                  
203684841     0                             0.0          0.0   
              1                             0.0          0.0   
              2                             0.0          0.0   
              3                             0.0          0.0   
              4                             0.0          0.0   

                 BEval_SP Video Infrastructure  \
SAVM_GROUP_ID                                    
203684841     0                            0.0   
              1                            0.0   
              2                            0.0   
              3                            0.0   
              4                            0.0   

                 BEval_SP Video Software and Solutions  BEval_Security  \
SAVM_GROUP_ID                                                            
203684841     0                               182560.0         25772.0   
              1                                    0.0         25453.0   
              2                               372422.0         26051.0   
              3                               372422.0         62115.0   
              4                               473800.0         62115.0   

                 BEval_Service Provider Routing  BEval_Services ASS_L1  \
SAVM_GROUP_ID                                                            
203684841     0                           153.0                    0.0   
              1                          4237.0                 3568.0   
              2                          4193.0                 3568.0   
              3                          4193.0 

In [124]:
df_aggregated_inserted.drop(['SAVM_GROUP_ID'],axis=1,inplace=True)

In [125]:
df_aggregated_inserted.reset_index(inplace=True)

In [126]:
df_aggregated_inserted.shape

(2788, 73)

In [127]:
df_aggregated_inserted.drop(['level_1'],axis=1,inplace=True)

In [128]:
df_aggregated_inserted.head()

,SAVM_GROUP_ID,year_quarter,Vert_Education- Public/Private,Vert_Energy/Utilities,Vert_Financial Services,Vert_Government,Vert_Health Care,Vert_Hospitality/Hotels & Leisure,Vert_Manufacturing,Vert_Media/Entertainment,...,BEval_Monitoring and Analytics,BEval_Other,BEval_SP Video Infrastructure,BEval_SP Video Software and Solutions,BEval_Security,BEval_Service Provider Routing,BEval_Services ASS_L1,BEval_Services TSS_L1,BEval_UNKNOWN,BEval_Wireless
0,203684841,201501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,182560.0,25772.0,153.0,0.0,708815.0,0.0,1007.0
1,203684841,201502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25453.0,4237.0,3568.0,776040.0,0.0,579895.0
2,203684841,201503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,372422.0,26051.0,4193.0,3568.0,857153.0,0.0,613614.0
3,203684841,201504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,372422.0,62115.0,4193.0,-92139.0,3706744.0,0.0,3025706.0
4,203684841,201601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,473800.0,62115.0,5314.0,-92139.0,4708604.0,0.0,3043443.0


In [129]:
beval_columns=[col for col in df_aggregated_inserted if col.startswith('BEval_')]

In [130]:
prod = 'BE_Reco_Prod'
x = [i for i in df_aggregated_inserted.columns.tolist() if i!='BE_Reco_Prod']
SAVM_Final = []
iloop=0
window_size = 3
for i in df_aggregated_inserted.SAVM_GROUP_ID.unique():
    savm_list = []
    Test_Df = df_aggregated_inserted.loc[df_aggregated_inserted.SAVM_GROUP_ID==i,:]
    for j in range(0,window_size+1):
        if(j<window_size):
            df_shift = Test_Df[x].shift(-j)
            df_shift.columns = [k+'t_'+str(j) for k in df_shift.columns]
            savm_list.append(df_shift)
        if(j == window_size):
            Shifted_Df = Test_Df[[prod]].shift(-j)
            Shifted_Df.columns = ['BE_Reco_Prod_t']
            savm_list.append(Shifted_Df)
    iloop+=1
    if(np.divmod(iloop,1000)[1]==0):
        print('completed '+str(iloop))
    df_IE2K = pd.concat(savm_list,axis=1)
    df_IE2K.dropna(inplace=True,axis=0)
    SAVM_Final.append(df_IE2K)

In [131]:
Final_Data = pd.concat(SAVM_Final,axis=0)
SAVM_collated =  Final_Data

In [132]:
SAVM_collated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2282 entries, 0 to 2780
Columns: 214 entries, SAVM_GROUP_IDt_0 to BE_Reco_Prod_t
dtypes: float64(211), int64(3)
memory usage: 3.7 MB


In [133]:
SAVM_collated['ACTIVE_3YEARSt_0'].describe()

count    2282.000000
mean        0.034107
std         0.083638
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         0.583333
Name: ACTIVE_3YEARSt_0, dtype: float64

In [134]:
SAVM_collated.replace([np.inf, -np.inf], np.nan,inplace=True)

In [135]:
SAVM_collated.fillna(0,inplace=True)

In [136]:
#beval_columns=[col for col in df_aggregated_inserted if col.startswith('BEval_')]

In [137]:
beval_columns_1=[col for col in SAVM_collated if col.startswith('BEval_')]
vr_columns_1=[col for col in SAVM_collated if col.startswith('Vert_')]
be_columns_1=[col for col in SAVM_collated if col.startswith('BE_')]


In [138]:
SAVM_collated[vr_columns_1]=SAVM_collated[vr_columns_1].astype(int)
SAVM_collated[be_columns_1]=SAVM_collated[be_columns_1].astype(int)
SAVM_collated[beval_columns_1]=SAVM_collated[beval_columns_1].astype(int)

In [139]:
#SAVM_collated = SAVM_collated.astype(int)

In [140]:
SAVM_collated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2282 entries, 0 to 2780
Columns: 214 entries, SAVM_GROUP_IDt_0 to BE_Reco_Prod_t
dtypes: float64(63), int32(148), int64(3)
memory usage: 2.5 MB


In [141]:
SAVM_collated['BE_Reco_Prod_t'] = np.where(SAVM_collated['BE_Reco_Prod_t']>=1,1,0)

In [142]:
SAVM_collated.to_csv('SAVM_collated_revised_DataCenter.csv')

In [143]:
SAVM_collated=pd.read_csv('SAVM_collated_revised_DataCenter.csv')

In [144]:
SAVM_collated.drop('Unnamed: 0',axis=1,inplace=True)

In [145]:
SAVM_collated['BE_Reco_Prod_t'].value_counts()

0    2118
1     164
Name: BE_Reco_Prod_t, dtype: int64

In [146]:
def add_qtr(x):
    qtr = int(str(int(x))[-1:])
    year = int(str(int(x))[:4])
    if(qtr==4):
        year = year+1
        yrqtr = str(year)+'01'
        yrqtr = int(yrqtr)
    else:
        yrqtr = int(x)+1    
    return(yrqtr)    

In [147]:
SAVM_collated['year_quartert_3'] = SAVM_collated['year_quartert_2'].apply(lambda x: add_qtr(x))

In [148]:
np.sort(SAVM_collated['year_quartert_3'].unique())

array([201204, 201301, 201302, 201303, 201304, 201401, 201402, 201403,
       201404, 201501, 201502, 201503, 201504, 201601, 201602, 201603,
       201604, 201701, 201702, 201703, 201704, 201801, 201802, 201803,
       201804, 201901, 201902, 201903, 201904, 202001, 202002, 202003],
      dtype=int64)

In [149]:
# train_period = [201204, 201301, 201302, 201303, 201304, 201401, 201402, 201403,
#        201404, 201501, 201502, 201503, 201504, 201601, 201602, 201603,
#        201604, 201701, 201702, 201703, 201704, 201801, 201802, 201803]
# val_period =  [201804, 201901, 201902, 201903]
# test_period = [201904, 202001, 202002, 202003]

In [150]:
train_period = [201204, 201301, 201302, 201303, 201304, 201401, 201402, 201403,
       201404, 201501, 201502, 201503, 201504, 201601, 201602, 201603,
       201604, 201701, 201702, 201703,201704, 201801,201802,201803,201804,201901,201902]
val_period =  [201903,201904]
test_period = [202001, 202002, 202003]

In [151]:
SAVM_collated_copy = SAVM_collated.copy()

In [152]:
#SAVM_collated= SAVM_collated_copy.copy()

In [153]:
# SAVM_collated.to_csv('SAVM_collated.csv')

In [154]:
droping = [i for i in SAVM_collated_copy.columns.tolist() if len(i.split('SAVM_GROUP_ID'))>1]

In [155]:
SAVM_collated.drop(droping,axis=1,inplace=True)

In [156]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
SAVM_collated[SAVM_collated.columns.difference(['BE_Reco_Prod_t','year_quartert_3'])]=scaler.fit_transform(SAVM_collated[SAVM_collated.columns.difference(['BE_Reco_Prod_t','year_quartert_3'])])

In [157]:
SAVM_collated['BE_Reco_Prod_t'].value_counts()

0    2118
1     164
Name: BE_Reco_Prod_t, dtype: int64

In [158]:
SAVM_collated.head()

,year_quartert_0,Vert_Education- Public/Privatet_0,Vert_Energy/Utilitiest_0,Vert_Financial Servicest_0,Vert_Governmentt_0,Vert_Health Caret_0,Vert_Hospitality/Hotels & Leisuret_0,Vert_Manufacturingt_0,Vert_Media/Entertainmentt_0,Vert_Professional Servicest_0,...,BEval_SP Video Infrastructuret_2,BEval_SP Video Software and Solutionst_2,BEval_Securityt_2,BEval_Service Provider Routingt_2,BEval_Services ASS_L1t_2,BEval_Services TSS_L1t_2,BEval_UNKNOWNt_2,BEval_Wirelesst_2,BE_Reco_Prod_t,year_quartert_3
0,0.426743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.056494,0.004362,0.000018,0.722870,0.012081,0.010744,0.014762,0,201504
1,0.428165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.056494,0.005722,0.000018,0.722344,0.046834,0.010744,0.067316,0,201601
2,0.429587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.071505,0.005722,0.000023,0.722344,0.059052,0.010744,0.067702,0,201602
3,0.431010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.056213,0.029572,0.000062,0.722850,0.060502,0.010744,0.083373,0,201603
4,0.568990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.067025,0.029574,0.000076,0.722850,0.061242,0.010744,0.085344,0,201604


In [159]:
SAVM_collated.drop(['year_quartert_0','year_quartert_1','year_quartert_2'],axis=1,inplace=True)

In [160]:
SAVM_collated.head()

,Vert_Education- Public/Privatet_0,Vert_Energy/Utilitiest_0,Vert_Financial Servicest_0,Vert_Governmentt_0,Vert_Health Caret_0,Vert_Hospitality/Hotels & Leisuret_0,Vert_Manufacturingt_0,Vert_Media/Entertainmentt_0,Vert_Professional Servicest_0,Vert_Retailt_0,...,BEval_SP Video Infrastructuret_2,BEval_SP Video Software and Solutionst_2,BEval_Securityt_2,BEval_Service Provider Routingt_2,BEval_Services ASS_L1t_2,BEval_Services TSS_L1t_2,BEval_UNKNOWNt_2,BEval_Wirelesst_2,BE_Reco_Prod_t,year_quartert_3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.056494,0.004362,0.000018,0.722870,0.012081,0.010744,0.014762,0,201504
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.056494,0.005722,0.000018,0.722344,0.046834,0.010744,0.067316,0,201601
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.071505,0.005722,0.000023,0.722344,0.059052,0.010744,0.067702,0,201602
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.056213,0.029572,0.000062,0.722850,0.060502,0.010744,0.083373,0,201603
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.067025,0.029574,0.000076,0.722850,0.061242,0.010744,0.085344,0,201604


In [161]:
Train = SAVM_collated.loc[SAVM_collated.year_quartert_3.isin(train_period),:]
Val=SAVM_collated.loc[SAVM_collated.year_quartert_3.isin(val_period),:]
Test = SAVM_collated.loc[SAVM_collated.year_quartert_3.isin(test_period),:]

In [162]:
Train.shape,Val.shape,Test.shape

((2015, 209), (165, 209), (102, 209))

In [163]:
Train.drop('year_quartert_3',axis=1,inplace=True)
Val.drop('year_quartert_3',axis=1,inplace=True)
Test.drop('year_quartert_3',axis=1,inplace=True)

In [164]:
y_train = Train['BE_Reco_Prod_t']
X_train = Train.drop('BE_Reco_Prod_t',axis=1)

In [165]:
y_val = Val['BE_Reco_Prod_t']
X_val = Val.drop('BE_Reco_Prod_t',axis=1)
y_test = Test['BE_Reco_Prod_t']
X_test = Test.drop('BE_Reco_Prod_t',axis=1)

In [166]:
X_train.shape,X_val.shape,X_test.shape,y_train.shape,y_val.shape,y_test.shape

((2015, 207), (165, 207), (102, 207), (2015,), (165,), (102,))

In [167]:
print(y_train.value_counts())
print(y_val.value_counts())
print(y_test.value_counts())

0    1942
1      73
Name: BE_Reco_Prod_t, dtype: int64
0    127
1     38
Name: BE_Reco_Prod_t, dtype: int64
1    53
0    49
Name: BE_Reco_Prod_t, dtype: int64


In [168]:
y_train.mean(),y_val.mean(),y_test.mean()

(0.036228287841191066, 0.23030303030303031, 0.5196078431372549)

In [169]:
# X_train_100 = X_train.head(100)
# X_train_100.to_csv('Train_EY_Colab.csv')

In [170]:
from collections import Counter
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
import tensorflow as tf
from keras.utils import to_categorical
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [171]:
#keras.__version__

In [172]:
import keras

In [173]:
cols=X_train.values.shape[1]

In [174]:
window_size=3

In [175]:
step_ = int(cols/window_size)

In [176]:
#Train_x,Train_y = Train_upsampled[x_].values.reshape((-1,window_size,step_)),Train_upsampled[[y]].values.reshape((Train_upsampled.shape[0],))
Train_x,Train_y = X_train.values.reshape((-1,window_size,step_)),y_train.values.reshape((Train.shape[0],))
Val_x,Val_y = X_val.values.reshape((-1,window_size,step_)),y_val.values.reshape((Val.shape[0],))
Test_x,Test_y = X_test.values.reshape((-1,window_size,step_)),y_test.values.reshape((Test.shape[0],))

In [177]:
Test_y.mean()

0.5196078431372549

In [178]:
print(Train_x.shape,Train_y.shape)
print(Val_x.shape,Val_y.shape)
print(Test_x.shape,Test_y.shape)

(2015, 3, 69) (2015,)
(165, 3, 69) (165,)
(102, 3, 69) (102,)


In [179]:
from keras.utils.np_utils import to_categorical

In [180]:
y_train_1 = to_categorical(Train_y)
y_val_1 = to_categorical(Val_y)
y_test_1 = to_categorical(Test_y)

In [181]:
from keras.regularizers import l2
from keras.optimizers import SGD

In [182]:
# class_weight={1: 1/(np.sum(Train_y) / len(Train_y)),0:1}

In [183]:
class_weight={1:20,0: 1}

In [184]:
class_weight

{1: 20, 0: 1}

In [185]:
neg=np.bincount(Train_y)[0]
pos=np.bincount(Train_y)[1]

In [186]:
initial_bias = np.log([pos/neg])

In [187]:
model=[]

In [188]:
input_ = keras.initializers.TruncatedNormal(mean=0., stddev=1.)
#output_bias = keras.initializers.Constant(initial_bias)
embedding_size=30
model=Sequential()
model.add(LSTM(200, input_shape=(Train_x.shape[1], Train_x.shape[2]),activation='relu',kernel_initializer=input_,kernel_regularizer=l2(0.01)))
#model.add(LSTM(200, input_shape=(Train_x.shape[1], Train_x.shape[2]),activation='relu'))
#model.add(Dense(2, activation='softmax',bias_initializer=output_bias))
model.add(Dense(2, activation='softmax'))
model.summary()
#opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='adamax',metrics=[keras.metrics.Precision(),keras.metrics.Recall()])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               216000    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 216,402
Trainable params: 216,402
Non-trainable params: 0
_________________________________________________________________


In [189]:
model.fit(Train_x, y_train_1, epochs=30, batch_size=2000, validation_data=(Val_x, y_val_1),verbose=2,shuffle=False,class_weight=class_weight)

Train on 2015 samples, validate on 165 samples
Epoch 1/30
 - 1s - loss: 429.7023 - precision_1: 0.4844 - recall_1: 0.4844 - val_loss: 427.1035 - val_precision_1: 0.3515 - val_recall_1: 0.3515
Epoch 2/30
 - 0s - loss: 427.2267 - precision_1: 0.4759 - recall_1: 0.4759 - val_loss: 424.4290 - val_precision_1: 0.3152 - val_recall_1: 0.3152
Epoch 3/30
 - 0s - loss: 424.4244 - precision_1: 0.3598 - recall_1: 0.3598 - val_loss: 421.6759 - val_precision_1: 0.3515 - val_recall_1: 0.3515
Epoch 4/30
 - 0s - loss: 421.6636 - precision_1: 0.3682 - recall_1: 0.3682 - val_loss: 418.6958 - val_precision_1: 0.3879 - val_recall_1: 0.3879
Epoch 5/30
 - 0s - loss: 418.7285 - precision_1: 0.4516 - recall_1: 0.4516 - val_loss: 415.6605 - val_precision_1: 0.4485 - val_recall_1: 0.4485
Epoch 6/30
 - 0s - loss: 415.7928 - precision_1: 0.5846 - recall_1: 0.5846 - val_loss: 412.6801 - val_precision_1: 0.5152 - val_recall_1: 0.5152
Epoch 7/30
 - 0s - loss: 412.9413 - precision_1: 0.6898 - recall_1: 0.6898 - val_lo

In [190]:
y_pred=model.predict_classes(Test_x)

In [191]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(list(y_test), y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(Test_y, y_pred))

0.5196078431372549
[[33 16]
 [33 20]]
              precision    recall  f1-score   support

           0       0.50      0.67      0.57        49
           1       0.56      0.38      0.45        53

    accuracy                           0.52       102
   macro avg       0.53      0.53      0.51       102
weighted avg       0.53      0.52      0.51       102



In [192]:
Test_y.mean()

0.5196078431372549

In [193]:
len(Test_y)

102

In [194]:
len(np.where(Test_y==0)[0])

49

In [195]:
len(np.where(Test_y==1)[0])

53

In [196]:
len(np.where(y_pred==0)[0])

66

In [197]:
len(np.where(y_pred==1)[0])

36

In [198]:
Train_y.mean()

0.036228287841191066

In [199]:
Val_y.mean()

0.23030303030303031

In [200]:
from sklearn.metrics import precision_recall_curve

In [201]:
from numpy import argmax

In [202]:
import matplotlib.pyplot as plt

In [203]:
# plt.hist(y_pred_test[1])

In [204]:
# len(np.where(Test_y==y_pred_)[0])

In [205]:
# df_Pred = pd.DataFrame({'Real':list(Test_y),'Pred':list(y_pred_)})

In [206]:
#df_Pred.loc[df_Pred.Real==1,:].Pred.sum()

In [207]:
Train_y.mean()

0.036228287841191066

In [208]:
y_pred_test=model.predict_proba(Test_x)

In [209]:
y_pred_test

array([[4.6748117e-01, 5.3251886e-01],
       [5.5260360e-01, 4.4739640e-01],
       [7.8925508e-01, 2.1074489e-01],
       [7.3062474e-01, 2.6937523e-01],
       [6.4921564e-01, 3.5078436e-01],
       [3.5163701e-01, 6.4836293e-01],
       [6.7046702e-01, 3.2953298e-01],
       [6.0997027e-01, 3.9002964e-01],
       [4.4872114e-01, 5.5127883e-01],
       [1.4970827e-01, 8.5029179e-01],
       [6.1468065e-02, 9.3853194e-01],
       [8.2738692e-01, 1.7261310e-01],
       [1.7681715e-01, 8.2318276e-01],
       [8.1841864e-02, 9.1815817e-01],
       [5.7391856e-02, 9.4260812e-01],
       [7.6425946e-01, 2.3574057e-01],
       [6.8051338e-01, 3.1948665e-01],
       [2.4581951e-01, 7.5418049e-01],
       [3.1801510e-01, 6.8198490e-01],
       [6.9588244e-02, 9.3041176e-01],
       [2.0016161e-01, 7.9983842e-01],
       [4.7265011e-01, 5.2734983e-01],
       [2.6304474e-01, 7.3695529e-01],
       [2.6015756e-01, 7.3984241e-01],
       [7.3771499e-02, 9.2622852e-01],
       [5.1432884e-01, 4.

In [229]:
#y_pred_prob = model.predict_proba(Train_x)
#precision, recall, thresholds = precision_recall_curve(Train_y, y_pred_prob[:,1])
#fscore = (2 * precision * recall) / (precision + recall)
#ix = argmax(fscore)
#t_max,f_max=thresholds[ix], fscore[ix]
y_pred_test=model.predict_proba(Test_x)
y_pred_ = np.where(y_pred_test[:,1]>0.5,1,0)
print(classification_report(Test_y, y_pred_))

              precision    recall  f1-score   support

           0       0.50      0.67      0.57        49
           1       0.56      0.38      0.45        53

    accuracy                           0.52       102
   macro avg       0.53      0.53      0.51       102
weighted avg       0.53      0.52      0.51       102



In [230]:
y_pred_ = np.where(y_pred_test[:,1]>0.2,1,0)
print(classification_report(Test_y, y_pred_))

              precision    recall  f1-score   support

           0       0.38      0.31      0.34        49
           1       0.46      0.55      0.50        53

    accuracy                           0.43       102
   macro avg       0.42      0.43      0.42       102
weighted avg       0.42      0.43      0.42       102



In [212]:
Train_y.mean()

0.036228287841191066

In [213]:
Val_y.mean()

0.23030303030303031

In [214]:
Test_y.mean()

0.5196078431372549

In [215]:
Val_y.mean()

0.23030303030303031

In [216]:
Val_y.shape

(165,)

In [217]:
Test_y.mean()

0.5196078431372549

In [218]:
pkl_filename = "DATACENTER_pickel.pkl"
with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file)

In [219]:
model.save('datacenter_pickel.h5')

In [220]:
Train.to_csv('Train_data_center.csv')

In [221]:
def get_lift_Chart(Test_y,y_pred_test):
    Prob_df = pd.DataFrame({'Bought':list(Test_y),'Prob':list(y_pred_test[:,1])})
    Prob_df['decile'] = pd.qcut(Prob_df['Prob'],10,labels=['1','2','3','4','5','6','7','8','9','10'])
    Prob_df['Not_Bought'] = 1-Prob_df['Bought']
    df1 = pd.pivot_table(data=Prob_df,index=['decile'],values=['Bought','Not_Bought','Prob'],
                     aggfunc={'Bought':[np.sum],
                              'Not_Bought':[np.sum],
                              'Prob' : [np.min,np.max]})
    df1.columns = ['Bought_Count','Not_Bought_Count','max_score','min_score']
    df1['Total_Cust'] = df1['Bought_Count']+df1['Not_Bought_Count']
    df2 = df1.sort_values(by='min_score',ascending=False)
    df2['Default_Rate'] = (df2['Bought_Count'] / df2['Total_Cust'])
    default_sum = df2['Bought_Count'].sum()
    non_default_sum = df2['Not_Bought_Count'].sum()
    df2['Default %'] = (df2['Bought_Count']/default_sum).apply('{0:.2%}'.format)
    df2['Non_Default %'] = (df2['Not_Bought_Count']/non_default_sum).apply('{0:.2%}'.format)
    df2['Lift'] = df2['Default_Rate']/Prob_df.Bought.mean()
    df2['Bought_cum_sum']= df2.Bought_Count.cumsum()
    df2['Not_Bought_cum_sum']= df2.Not_Bought_Count.cumsum()
    df2['Total_Cust_cum_sum']= df2.Total_Cust.cumsum()
    df2['Cum_Default_Rate'] = df2['Bought_cum_sum']/df2['Total_Cust_cum_sum']
    df2['Lift_Cum'] = df2['Cum_Default_Rate']/Prob_df.Bought.mean()
    return(df2)

In [222]:
lift_chart = get_lift_Chart(Test_y,y_pred_test)

In [223]:
lift_chart['BE']='Data_Center'

In [224]:
lift_chart.reset_index(inplace=True)

In [225]:
lift_chart['Test_Mean']=Test_y.mean()

In [226]:
lift_chart.head()

,decile,Bought_Count,Not_Bought_Count,max_score,min_score,Total_Cust,Default_Rate,Default %,Non_Default %,Lift,Bought_cum_sum,Not_Bought_cum_sum,Total_Cust_cum_sum,Cum_Default_Rate,Lift_Cum,BE,Test_Mean
0,10,7,4,0.996128,0.823089,11,0.636364,13.21%,8.16%,1.224700,7,4,11,0.636364,1.224700,Data_Center,0.519608
1,9,4,6,0.799941,0.665897,10,0.400000,7.55%,12.24%,0.769811,11,10,21,0.523810,1.008086,Data_Center,0.519608
2,8,7,3,0.648363,0.532519,10,0.700000,13.21%,6.12%,1.347170,18,13,31,0.580645,1.117468,Data_Center,0.519608
3,7,4,6,0.532072,0.443921,10,0.400000,7.55%,12.24%,0.769811,22,19,41,0.536585,1.032674,Data_Center,0.519608
4,6,2,8,0.408478,0.329533,10,0.200000,3.77%,16.33%,0.384906,24,27,51,0.470588,0.905660,Data_Center,0.519608


In [227]:
lift_chart.to_csv('DataCenter_lifts.csv')

In [228]:
df_temp_final[['year','shifted_savm_count']].drop_duplicates().to_csv('datacenter_savm_count.csv')